<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Storage Test</span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Workspace Initialization 

In [0]:
Workspace.Initialize(x => x.FromSource(DataSource)
                           .DisableInitialization<RawVariable>()
                           .DisableInitialization<IfrsVariable>());

# Test IfrsVariables Query

In [0]:
var gic = "DT1.1";
var gric = "DTR1.1";
var reportingNode = "CH";
var scenario = (string)null;

In [0]:
//Define partition
var args = new ImportArgs(reportingNode, 2021, 3, Periodicity.Quarterly, scenario, ImportFormats.Actual);
var previousArgs = new ImportArgs(reportingNode, 2020, 12, Periodicity.Quarterly, scenario, ImportFormats.Actual);
var partition = new PartitionByReportingNodeAndPeriod { Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(args)),
                                                        ReportingNode = reportingNode, 
                                                        Scenario = scenario, 
                                                        Year = args.Year,
                                                        Month = args.Month };
var previousPeriodPartition = new PartitionByReportingNodeAndPeriod { Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(previousArgs)),
                                                                      ReportingNode = reportingNode, 
                                                                      Scenario = scenario, 
                                                                      Year = previousArgs.Year,
                                                                      Month = previousArgs.Month };
await DataSource.UpdateAsync(new[]{partition, previousPeriodPartition});
await DataSource.CommitAsync();

In [0]:
public async Task<ActivityLog> CheckIfrsVariablesFromImportStorageAsync(string importFormat, IEnumerable<IfrsVariable> inputDataSetForWorkspace, IEnumerable<IfrsVariable> inputDataSetForDataSource, IEnumerable<IfrsVariable> ivsBenchmark)
{
    Activity.Start();
    //Prepare Workspace and DataSource
    await Workspace.UpdateAsync<IfrsVariable>(inputDataSetForWorkspace);
    await DataSource.UpdateAsync<IfrsVariable>(inputDataSetForDataSource);
    
    //Set up import storage and test universe
    var testStorage = new ImportStorage(args with {ImportFormat = importFormat}, DataSource, Workspace);
    await testStorage.InitializeAsync();
    
    var ivs = testStorage.IfrsVariablesByImportIdentity.SelectMany(x => x.Value);
    
    //Clean up Workspace
    //Workspace.Reset();    
    await Workspace.DeleteAsync<IfrsVariable>(inputDataSetForWorkspace);    
    await DataSource.DeleteAsync<IfrsVariable>(inputDataSetForDataSource);    
    
    var errors = new List<string>();
    
    var extraVariablesInStorage = ivs.Except(ivsBenchmark).Select(x => x.ToIdentityString()).ToArray();
    if(extraVariablesInStorage.Any()) errors.Add( $"IfrsVariables in the storage contain the following items that are not present in the benchmark:\n{string.Join("\n",extraVariablesInStorage)}." );
       
    var extraVariablesInBenchmark = ivsBenchmark.Except(ivs).Select(x => x.ToIdentityString()).ToArray();
    if(extraVariablesInBenchmark.Count() > 0) errors.Add( $"IfrsVariables in the benchmark contain the following items that are not present in the storage:\n{string.Join("\n",extraVariablesInBenchmark)}." );
        
    //find duplicates in storage
    var ivsByIdentityString = ivs.GroupBy(x => x.ToIdentityString()).Where(x => x.Count() > 1).Select(x => x.Key);
    if(ivsByIdentityString.Any()) errors.Add($"IfrsVariables in the storage have duplicated items for:\n{string.Join("\n",ivsByIdentityString)}.");
    
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

In [0]:
//There is a previous year, and it's the first time Actuals are imported for the current year
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = gic, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};
var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 100.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -10.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 15.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -10.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 15.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
//There is a previous year, and it's the second time Actuals are imported for the current year (reimport)
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = gic, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};
var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 100.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = 150.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 200.0},
    basicIfrsVariable with {AocType = AocTypes.EOP, Value = 450.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -15.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = -20.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -15.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = -20.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
//There is a previous year, Cash flows were already imported and Actuals are imported for the first time
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = gic, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.I, EstimateType = EstimateTypes.BE};
var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = 100.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.IA, Value = 1500.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = 2500.0},
    basicIfrsVariable with {AocType = AocTypes.EOP, Value = 5000.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CF, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = -15.0},
    basicIfrsVariable with {AocType = AocTypes.WO, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = -20.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.IA, Value = 1500.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = 2500.0},
    basicIfrsVariable with {AocType = AocTypes.EOP, Value = 5000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.AA, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CF, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = -15.0},
    basicIfrsVariable with {AocType = AocTypes.WO, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = -20.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
//There is a previous year, Cash flows and Actuals were already imported and Actuals are imported again
var basicAdvanceActualIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = gic, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};
var basicBeIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = gic, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.I, EstimateType = EstimateTypes.BE};
var inputDataSetForDataSource = new IfrsVariable[]{
    basicBeIfrsVariable            with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 1000.0},
    basicBeIfrsVariable            with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 1000.0},
    basicBeIfrsVariable            with {AocType = AocTypes.IA, Value = 1500.0},
    basicBeIfrsVariable            with {AocType = AocTypes.CF, Value = 2500.0},
    basicBeIfrsVariable            with {AocType = AocTypes.EOP, Value = 5000.0},
    basicAdvanceActualIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 100.0},
    basicAdvanceActualIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.CF, Value = 150.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.WO, Value = 200.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.EOP, Value = 450.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.CF, Value = -15.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.WO, Value = -20.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicBeIfrsVariable            with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 1000.0},
    basicBeIfrsVariable            with {AocType = AocTypes.IA, Value = 1500.0},
    basicBeIfrsVariable            with {AocType = AocTypes.CF, Value = 2500.0},
    basicBeIfrsVariable            with {AocType = AocTypes.EOP, Value = 5000.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.CF, Value = -15.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.WO, Value = -20.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
//There is a previous year for Primary Scope (GRIC10) 
//There is a previous and current year for Secondary Scope (GIC10), 
//It's the first time Actuals are imported for the current year

var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = gric, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};
var inputDataSetForDataSource = new IfrsVariable[]{
    //Year
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 100.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, DataNode = gic, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.C, Value = 1000.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.CL,  DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.C, Value = 666.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 1000.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.CL,  DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 666.0},
    //Year -1
    basicIfrsVariable with {AocType = AocTypes.IA, Novelty = Novelties.I, DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 10.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Novelty = Novelties.I, DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 10.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -10.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 15.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    //From previous Period
    //Actuals
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, DataNode = gic, EstimateType = EstimateTypes.DA, Value = 1000.0},
    //Cash flow
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.C, Value = 1000.0},
    
    //From current Period
    //from DB
    basicIfrsVariable with {AocType = AocTypes.IA, Novelty = Novelties.I, DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 10.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Novelty = Novelties.I, DataNode = gic, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 10.0},
    
    //from workspace
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -10.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 15.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

# Data Node Parameter related logic

## Data Preparation

In [0]:
//Define partition
var args = new ImportArgs("CH", 2021, 3, Periodicity.Quarterly, null, ImportFormats.Cashflow);

var reportingNodePartition = DataSource.Query<PartitionByReportingNode>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode && x.Scenario == args.Scenario);
if(reportingNodePartition == null) ApplicationMessage.Log(Error.PartitionNotFound);


var currentPartition = DataSource.Query<PartitionByReportingNodeAndPeriod>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode && x.Year == args.Year && 
                                                                                    x.Month == args.Month && x.Scenario == args.Scenario);
if(currentPartition == null) ApplicationMessage.Log(Error.PartitionNotFound);


var previousPeriodPartition = Workspace.Query<PartitionByReportingNodeAndPeriod>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode && x.Year == args.Year - 1 && 
                                                                                    x.Month == MonthInAYear && x.Scenario == args.Scenario);
if(previousPeriodPartition == null) ApplicationMessage.Log(Error.PartitionNotFound);

In [0]:
var sampleGic = new GroupOfInsuranceContract(){Portfolio = "P1"};
var inputDataGic = new GroupOfInsuranceContract[]{ sampleGic with {SystemName = "Gross1", LiabilityType = LiabilityTypes.LRC},
                                                   sampleGic with {SystemName = "Gross2", LiabilityType = LiabilityTypes.LRC},
                                                   sampleGic with {SystemName = "Gross3", LiabilityType = LiabilityTypes.LRC},
                                                 };

var sampleGric = new GroupOfReinsuranceContract(){Portfolio = "ReP1"};
var inputDataGric = new GroupOfReinsuranceContract[]{ sampleGric with {SystemName = "Reins1", LiabilityType = LiabilityTypes.LRC},
                                                      sampleGric with {SystemName = "Reins2", LiabilityType = LiabilityTypes.LRC},
                                                      sampleGric with {SystemName = "Reins3", LiabilityType = LiabilityTypes.LRC},
                                                 };
var sampleDnState = new DataNodeState {Partition = reportingNodePartition.Id, Year = args.Year, Month = args.Month, State = State.Active};
var inputDataState = new DataNodeState[]{ sampleDnState with {DataNode = "Gross1"} ,
                                         sampleDnState with {DataNode = "Gross2"} ,
                                         sampleDnState with {DataNode = "Gross3"} ,
                                         sampleDnState with {DataNode = "Reins1"} ,
                                         sampleDnState with {DataNode = "Reins2"} ,
                                         sampleDnState with {DataNode = "Reins3"} ,
                                        };

var samplePreviousDnParam = new InterDataNodeParameter {Partition = reportingNodePartition.Id, ReinsuranceCoverage = 1, Year = args.Year -1, Month = args.Month};
var sampleCurrentDnParam = new InterDataNodeParameter {Partition = reportingNodePartition.Id, ReinsuranceCoverage = 1, Year = args.Year, Month = args.Month};
var inputDataParameter = new InterDataNodeParameter[]{
                                                      samplePreviousDnParam with {DataNode = "Gross3", LinkedDataNode = "Reins3", ReinsuranceCoverage = 1},
                                                      samplePreviousDnParam with {DataNode = "Gross1", LinkedDataNode = "Reins1", ReinsuranceCoverage = 0.1},
                                                      samplePreviousDnParam with {DataNode = "Gross1", LinkedDataNode = "Reins2", ReinsuranceCoverage = 0.2},
                                                      samplePreviousDnParam with {DataNode = "Gross2", LinkedDataNode = "Reins2", ReinsuranceCoverage = 0.3},
                                                      sampleCurrentDnParam with {DataNode = "Gross1", LinkedDataNode = "Reins1", ReinsuranceCoverage = 0.5},
                                                      sampleCurrentDnParam with {DataNode = "Gross1", LinkedDataNode = "Reins2", ReinsuranceCoverage = 0.6},
                                                      sampleCurrentDnParam with {DataNode = "Gross2", LinkedDataNode = "Reins2", ReinsuranceCoverage = 0.7},
                                                      sampleCurrentDnParam with {DataNode = "Gross3", LinkedDataNode = "Reins3", ReinsuranceCoverage = 1.0},
                                        };

var sampleRawVar = new RawVariable{AmountType = AmountTypes.PR, AocType = AocTypes.CL, Novelty = Novelties.C, Partition = currentPartition.Id};

In [0]:
public async Task PrepareWorkspaceDataNodes()
{
    await Workspace.UpdateAsync<GroupOfInsuranceContract>(inputDataGic);
    await Workspace.UpdateAsync<GroupOfReinsuranceContract>(inputDataGric);
    await Workspace.UpdateAsync<DataNodeState>(inputDataState);
    await Workspace.UpdateAsync<InterDataNodeParameter>(inputDataParameter);
}

In [0]:
public async Task CleanWorkspaceDataNodes()
{
    await Workspace.DeleteAsync<GroupOfInsuranceContract>(inputDataGic);
    await Workspace.DeleteAsync<GroupOfReinsuranceContract>(inputDataGric);
    await Workspace.DeleteAsync<DataNodeState>(inputDataState);
    await Workspace.DeleteAsync<InterDataNodeParameter>(inputDataParameter);
}

## Get Underlying Gic

In [0]:
public async Task<ActivityLog> CheckGetUnderlyingGicsAsync(RawVariable[] inputDataVariable, Dictionary<string,IEnumerable<string>> underlyingGicBm)
{
    Activity.Start();
    var errors = new List<string>();
    
    await PrepareWorkspaceDataNodes();
    await Workspace.UpdateAsync<RawVariable>(inputDataVariable);
    var testStorage = new ImportStorage(args, DataSource, Workspace);
    await testStorage.InitializeAsync();
    
    var primaryScopeDn = testStorage.DataNodesByImportScope[ImportScope.Primary];
    
    foreach (var dn in primaryScopeDn)
    {
        var id = new ImportIdentity(){DataNode = dn};
        if ( underlyingGicBm[dn].Except(testStorage.GetUnderlyingGic(id)).Count() != 0 )
            errors.Add( $"Underlying Gics for DataNode {dn} not matching with BM. Computed: \n{string.Join("\n",testStorage.GetUnderlyingGic(id))} \n Expected : \n{string.Join("\n",underlyingGicBm[dn])}" );
    }
    
    await Workspace.DeleteAsync<RawVariable>(Workspace.Query<RawVariable>());
    await CleanWorkspaceDataNodes();
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           sampleRawVar with {DataNode = "Reins2"},
                                         };
var underlyingGicBm = new Dictionary<string,IEnumerable<string>>(){
    {"Reins1",new string[]{"Gross1"}},
    {"Reins2",new string[]{"Gross1","Gross2"}},
};

var activity = await CheckGetUnderlyingGicsAsync(inputDataVariable, underlyingGicBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins2"},
                                         };
var underlyingGicBm = new Dictionary<string,IEnumerable<string>>(){
    {"Reins2",new string[]{"Gross1","Gross2"}},
};

var activity = await CheckGetUnderlyingGicsAsync(inputDataVariable, underlyingGicBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Get Reinsurance Coverage

In [0]:
public async Task<ActivityLog> CheckGetReinsuranceCoverageAsync(RawVariable[] inputDataVariable,  
                                                   Dictionary<(string, string),double> reinsCovBoPBm, 
                                                   Dictionary<(string, string),double> reinsCovEoPBm)
{
    Activity.Start();
    var errors = new List<string>();

    await PrepareWorkspaceDataNodes();
    await Workspace.UpdateAsync<RawVariable>(inputDataVariable);
    var testStorage = new ImportStorage(args, DataSource, Workspace);
    await testStorage.InitializeAsync();
    
    var primaryScopeDn = testStorage.DataNodesByImportScope[ImportScope.Primary];
    
    foreach (var dn in primaryScopeDn)
    {
        var aocTypes = new AocStep[]{ new AocStep(AocTypes.BOP, Novelties.I), new AocStep(AocTypes.RCU, Novelties.I) };
        foreach (var aoc in aocTypes)
        {
            var bm = aoc.AocType == AocTypes.BOP ? reinsCovBoPBm : reinsCovEoPBm;
            var id = new ImportIdentity(){DataNode = dn, AocType = aoc.AocType, Novelty = aoc.Novelty};
            
            var computedReinsCov = testStorage.GetUnderlyingGic(id)
                                              .Select(gic => (g: gic, value: testStorage.GetReinsuranceCoverage(id, gic)))
                                              .ToDictionary(x => (dn,x.g), x => x.value);
            
            if( bm.Keys.Where(x => x.Item1 == dn).Except(computedReinsCov.Keys).Count() != 0)
               errors.Add( $"Gric-Gic links not matching with BM for DataNode {dn} and AocType {aoc.AocType}. \n Computed: \n{string.Join("\n",computedReinsCov.Keys)} \n Expected: \n{string.Join("\n",bm.Keys)}" );
            
            foreach (var reinsCov in computedReinsCov)
            {
                var bmKvp = bm.Single(x => x.Key.Item1 == reinsCov.Key.Item1 && x.Key.Item2 == reinsCov.Key.Item2); 
                if( Math.Abs(bmKvp.Value - reinsCov.Value) > Precision )
                    errors.Add( $"{dn}-{reinsCov.Key.Item2} Reinsurance Coverage not matching with BM for AocType {aoc.AocType}: \n Computed: {reinsCov.Value} \n Expected: {bmKvp.Value}");
            }
        }
    }
    
    await Workspace.DeleteAsync<RawVariable>(Workspace.Query<RawVariable>());
    await CleanWorkspaceDataNodes();
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           sampleRawVar with {DataNode = "Reins2"},
                                         };
var reinsCovBoPBm = new Dictionary<(string, string),double>(){
    {("Reins1","Gross1"),0.1},
    {("Reins2","Gross1"),0.2},
    {("Reins2","Gross2"),0.3},
};

var reinsCovEoPBm = new Dictionary<(string, string),double>(){
    {("Reins1","Gross1"),0.5},
    {("Reins2","Gross1"),0.6},
    {("Reins2","Gross2"),0.7},
};

var activity = await CheckGetReinsuranceCoverageAsync(inputDataVariable, reinsCovBoPBm, reinsCovEoPBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Primary and Secondary Scope

In [0]:
public async Task<ActivityLog> CheckSecondaryScopeAsync(RawVariable[] inputDataVariable, string[] primaryScopeBm, string[] secondaryScopeBm)
{
    Activity.Start();
    var errors = new List<string>();
    
    await PrepareWorkspaceDataNodes();
    await Workspace.UpdateAsync<RawVariable>(inputDataVariable);
    var testStorage = new ImportStorage(args, DataSource, Workspace);
    await testStorage.InitializeAsync();
    
    var activeDn = (await Workspace.Query<DataNodeState>().ToArrayAsync()).Select(x => x.DataNode);
    
    var primaryScopeDn = testStorage.DataNodesByImportScope[ImportScope.Primary];
    
    foreach (var dn in activeDn)
    {
        //PrimaryScope
        if ( primaryScopeBm.Contains(dn) && !primaryScopeDn.Contains(dn))
             errors.Add( $"DataNode {dn} is not added to the primary scope." );
        if( !primaryScopeBm.Contains(dn) && primaryScopeDn.Contains(dn))
             errors.Add( $"DataNode {dn} is added to the primary scope but should have not." );
        
        //SecondaryScope
        if ( secondaryScopeBm.Contains(dn) && !testStorage.IsSecondaryScope(dn))
             errors.Add( $"DataNode {dn} is not added to the secondary scope." );
        if( !secondaryScopeBm.Contains(dn) && testStorage.IsSecondaryScope(dn))
             errors.Add( $"DataNode {dn} is added to the secondary scope but should have not." );
    }
    
    await Workspace.DeleteAsync<RawVariable>(Workspace.Query<RawVariable>());
    await CleanWorkspaceDataNodes();
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           sampleRawVar with {DataNode = "Reins2"},
                                         };
var primaryScopeBm = new string[]{"Reins1", "Reins2"};
var secondaryScopeBm = new string[]{"Gross1", "Gross2"};
var activity = await CheckSecondaryScopeAsync(inputDataVariable, primaryScopeBm, secondaryScopeBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           };
var primaryScopeBm = new string[]{"Reins1"};
var secondaryScopeBm = new string[]{"Gross1"};
var activity = await CheckSecondaryScopeAsync(inputDataVariable, primaryScopeBm, secondaryScopeBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           sampleRawVar with {DataNode = "Gross1"},
                                           };
var primaryScopeBm = new string[]{"Reins1","Gross1","Reins2"};
var secondaryScopeBm = new string[]{"Gross2"};
var activity = await CheckSecondaryScopeAsync(inputDataVariable, primaryScopeBm, secondaryScopeBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);